In [29]:
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
from pprint import pprint
import pandas as pd

# In[2]:


client_credentials_manager = SpotifyClientCredentials(client_id="d63bed18ebe04b1ea09d8fb466370112",
    client_secret="83097a8de86940ed81505b3abbb077f5")
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


# https://newsroom.spotify.com/2021-12-01/what-the-world-streamed-most-in-2021/

# In[3]:


top_5_artists = ["Bad Bunny", "Taylor Swift" "BTS","Drake", "Justin Bieber"]

In [68]:
top_1000_artist = pd.read_csv("top_artist.tsv",sep="\t")
top_artists = list(top_1000_artist.Artist)

In [34]:
def get_all_track_artist(artist_id, exclude_remix=True, limit_num_album=50):
    # Get all albums
    albums = sp.artist_albums(artist_id, country="US", album_type='single', limit=limit_num_album, market="US")
    albums = {x['id']:x['name'] for x in albums['items']}
    if exclude_remix:
        albums = {key:value for (key,value) in albums.items() if "remix" not in value.lower()}
    # Filter duplicate
    albums_nondup = dict()
    for k, v in albums.items():
        if v not in albums_nondup.values():
            albums_nondup[k] = v
    # Get track from albums
    all_tracks = {}
    for alb_key in albums_nondup:
        alb_track = sp.album_tracks(alb_key)["items"]
        alb_track = {x['id']:x['name'] for x in alb_track}
        for k, v in alb_track.items():
            if v not in all_tracks.values():
                all_tracks[k] = v
    return all_tracks


In [59]:
def getTrackFeatures(id, sp):
  meta = sp.track(id)
  features = sp.audio_features(id)

  # Meta
  name = meta['name']
  album = meta['album']['name']
  artist = meta['album']['artists'][0]['name']
  release_date = meta['album']['release_date']
  length = meta['duration_ms']
  popularity = meta['popularity']

  # Features
  acousticness = features[0]['acousticness']
  danceability = features[0]['danceability']
  energy = features[0]['energy']
  instrumentalness = features[0]['instrumentalness']
  liveness = features[0]['liveness']
  loudness = features[0]['loudness']
  speechiness = features[0]['speechiness']
  tempo = features[0]['tempo']
  time_signature = features[0]['time_signature']

  track = [name, album, artist, release_date, length, popularity, 
    danceability, acousticness, energy, instrumentalness, 
    liveness, loudness, speechiness, tempo, time_signature]
  return track  

In [67]:
pd.read_csv("dummy_data.csv").drop(columns=["id"]).columns.__len__()

15

In [69]:
# top_artists = ["Bad Bunny", "Taylor Swift", "BTS","Drake", "Justin Bieber", "Mohit Chauhan", "Amit Trivedi", "Shubh", "AP Dhillon"]
# tt = ["Bad Bunny"]

In [63]:
get_artist_id = lambda x: x["artists"]["items"][0]["id"]
get_track_id = lambda x: x["id"]

In [70]:
result = []
for search_str in top_artists[:500]:
    print (f"Searching for {search_str}")
    try:
        artist_search = sp.search(search_str,type="artist",limit=1)
        artist_id = get_artist_id(artist_search)
    except:
        print ("-- Error --")
        continue
    top_ten_tracks_artist = sp.artist_top_tracks(artist_id)["tracks"]
    for track in top_ten_tracks_artist:
        track_id = get_track_id(track)
        # df_row_track = pd.DataFrame.from_dict(getTrackFeatures(track_id,sp),orient="index").T
        df_row_track = getTrackFeatures(track_id,sp)
        result.append(df_row_track)

Searching for Drake
Searching for Bad Bunny
Searching for Ed Sheeran
Searching for The Weeknd
Searching for Taylor Swift
Searching for Justin Bieber
Searching for Ariana Grande
Searching for Eminem
Searching for Post Malone
Searching for BTS
Searching for J Balvin
Searching for Kanye West
Searching for Billie Eilish
Searching for Juice WRLD
Searching for Coldplay
Searching for Dua Lipa
Searching for Imagine Dragons
Searching for XXXTENTACION
Searching for Ozuna
Searching for Khalid
Searching for Rihanna
Searching for Maroon 5
Searching for Travis Scott
Searching for David Guetta
Searching for Shawn Mendes
Searching for Bruno Mars
Searching for Daddy Yankee
Searching for Calvin Harris
Searching for Kendrick Lamar
Searching for Sam Smith
Searching for Queen
Searching for Beyoncé
Searching for Future
Searching for One Direction
Searching for The Chainsmokers
Searching for Lady Gaga
Searching for J. Cole
Searching for Nicki Minaj
Searching for Adele
Searching for Anuel AA
Searching for Hal

In [73]:
pd.DataFrame(result, columns=[
    "name", "album", "artist", "release_date", "length", "popularity", 
    "danceability", "acousticness", "energy", "instrumentalness", 
    "liveness", "loudness", "speechiness", "tempo", "time_signature"
    ]).drop_duplicates().to_csv("top_500_artists_deduplicate.csv", index=False)